In [1]:
import ollama 

In [2]:
inventory_db = {
        "laptop": {"stock": 5, "base_price": 1200},
        "monitor": {"stock": 0, "base_price": 300},
        "keyboard": {"stock": 25, "base_price": 80}
    }

In [3]:
#DFINING TOOL 
# TOOL 1:CHECKING DATABASE
def check_inventory(product_name):
    product_name = product_name.lower()
    
    if product_name in inventory_db:
        return inventory_db[product_name]
    
    return {"stock": 0, "base_price": None}

In [4]:
# TOOL 2: Business Logic for Discounts
def calculate_loyalty_discount(base_price, years_as_customer):
    discount = min(years_as_customer * 0.05, 0.30)  # max 30%
    final_price = base_price * (1 - discount)
    return round(final_price, 2)

In [5]:
# 1. Map the functions so the script can call them by name
available_functions = {
    'check_inventory': check_inventory,
    'calculate_loyalty_discount': calculate_loyalty_discount,
}

In [6]:
#DEFINING SCHEMA
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_inventory",
            "description": "Get stock and price for a product",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {"type": "string"}
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_loyalty_discount",
            "description": "Calculate final price based on loyalty years",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_price": {"type": "number"},
                    "years_as_customer": {"type": "integer"}
                },
                "required": ["base_price", "years_as_customer"]
            }
        }
    }
]

In [7]:
tools

[{'type': 'function',
  'function': {'name': 'check_inventory',
   'description': 'Get stock and price for a product',
   'parameters': {'type': 'object',
    'properties': {'product_name': {'type': 'string'}},
    'required': ['product_name']}}},
 {'type': 'function',
  'function': {'name': 'calculate_loyalty_discount',
   'description': 'Calculate final price based on loyalty years',
   'parameters': {'type': 'object',
    'properties': {'base_price': {'type': 'number'},
     'years_as_customer': {'type': 'integer'}},
    'required': ['base_price', 'years_as_customer']}}}]

In [8]:
#“I am a customer for 5 years. What will be the final price of a laptop?"
#"I want to buy an iPhone. Can you check stock?"
message=[
        {"role": "user", "content":"I am a customer for 5 years. What will be the final price of a laptop?"}
    ]

#CALLING llM
response = ollama.chat(
    model="gemma3:4b",
    messages=message
   ,
    tools=tools
)

print(response)

ResponseError: registry.ollama.ai/library/gemma3:4b does not support tools (status code: 400)

In [9]:
response["message"]

NameError: name 'response' is not defined

In [10]:
# Check if the model asked to use any tools
tool_calls = response['message'].get('tool_calls')

if tool_calls:
    for tool_call in tool_calls:
        tool_name = tool_call['function']['name']
        tool_args = tool_call['function']['arguments']

        # Get the actual Python function
        function_to_call = available_functions[tool_name]

        # Run the function with given arguments
        result = function_to_call(**tool_args)

        # Add the model's tool request to the conversation
        message.append(response['message'])

        # Add the tool's result back to the conversation
        message.append({
            'role': 'tool',
            'content': str(result)
        })

message

NameError: name 'response' is not defined

In [ ]:
# Ask the model to generate the final answer
final_response = ollama.chat(
    model='gemma3:4b',
    messages=message
)

print(final_response['message']['content'])